In [1]:
import torch
import torch.nn as nn

class CustomLayer(nn.Module):
    def __init__(self, n_features):
        super(CustomLayer, self).__init__()
        self.n_features = n_features
        # Initialize the interaction tensor as a learnable parameter
        self.interaction_tensor = nn.Parameter(torch.randn(n_features, n_features, n_features))

    def forward(self, x):
        # x is expected to be of size [batch, length, n_features]
        batch, length, n_features = x.size()

        # Process each [n_features] vector across batch and length
        output = x.new_empty(batch, length, n_features)
        for b in range(batch):
            for l in range(length):
                state_tensor = x[b, l, :]  # Shape: [n_features]
                # Step 1: Multiply state tensor by interaction tensor to get transition tensor
                # We manually implement the multiplication to match your operation
                transition_tensor = torch.einsum('i,ijk->jk', state_tensor, self.interaction_tensor)
                # Step 2: Multiply the transition tensor by the state tensor
                # Resulting shape: [n_features]
                output[b, l, :] = torch.matmul(transition_tensor, state_tensor)

        return output


In [2]:
import torch
import torch.nn as nn

class CustomLayerVectorized(nn.Module):
    def __init__(self, n_features):
        super(CustomLayerVectorized, self).__init__()
        self.n_features = n_features
        # Initialize the interaction tensor as a learnable parameter
        self.interaction_tensor = nn.Parameter(torch.randn(n_features, n_features, n_features))

    def forward(self, x):
        # x is expected to be of size [batch, length, n_features]
        batch, length, n_features = x.size()

        # Pre-allocate output tensor
        output = x.new_empty(batch, length, n_features)

        # Loop over length, but vectorize over the batch
        for l in range(length):
            # Extract all vectors at position l across all batches
            state_tensor = x[:, l, :]  # Shape: [batch, n_features]
            
            # Vectorized operation for all batches
            # Step 1: Calculate the transition tensor
            # Since we cannot directly use einsum for batched operation in this specific scenario,
            # we manually broadcast and multiply to achieve the intended result.
            # This involves expanding dimensions to enable broadcasting.
            state_tensor_expanded = state_tensor.unsqueeze(1).expand(-1, n_features, -1)  # Shape: [batch, n_features, n_features]
            interaction_tensor_expanded = self.interaction_tensor.unsqueeze(0).expand(batch, -1, -1, -1)  # Shape: [batch, n_features, n_features, n_features]
            # Multiply and sum over the last dimension to get the transition tensor
            transition_tensor = torch.einsum('bik,bijk->bij', state_tensor_expanded, interaction_tensor_expanded)
            
            # Step 2: Multiply the transition tensor by the state tensor to get the output
            output[:, l, :] = torch.einsum('bij,bj->bi', transition_tensor, state_tensor)

        return output



In [3]:
# n_features = 16
# batch, length, dim = 2, 64, n_features
# x = torch.randn(batch, length, dim)
# model = CustomLayerVectorized(
#     n_features = n_features
# )
# y = model(x)

# print(x.shape)
# assert y.shape == x.shape

In [4]:
import torch

def torch_randnorm(size, dim=0):
    # Generate a random tensor
    rand_tensor = torch.rand(size)
    
    # Normalize along the specified dimension
    sum_along_dim = torch.sum(rand_tensor, dim=dim, keepdim=True)
    normalized_tensor = rand_tensor / sum_along_dim
    
    return normalized_tensor

# Example usage
normalized_tensor = torch_randnorm([5,6], dim=1)
print(normalized_tensor)
print(normalized_tensor.sum(dim=1))

tensor([[0.0587, 0.1728, 0.1957, 0.1748, 0.1656, 0.2324],
        [0.1299, 0.1553, 0.2395, 0.0392, 0.2341, 0.2018],
        [0.0315, 0.0098, 0.1116, 0.2891, 0.3457, 0.2124],
        [0.1019, 0.2389, 0.2770, 0.0018, 0.1091, 0.2714],
        [0.0646, 0.1299, 0.2299, 0.1535, 0.0980, 0.3241]])
tensor([1., 1., 1., 1., 1.])


In [5]:
class InteractionModule(nn.Module):
    def __init__(self, n_features):
        super(InteractionModule, self).__init__()
        self.n_features = n_features
        # Initialize a set of interaction tensors, one for the state tensor and one for each column of the transition tensor
        self.interaction_tensors = nn.ParameterList([nn.Parameter(torch_randnorm([n_features, n_features, n_features], dim = 1)) for _ in range(n_features + 1)])

    def forward(self, state_tensor, previous_transition_tensor):
        # Get batch size
        batch = state_tensor.shape[0]
        # Assuming previous_transition_tensors is a list of transition tensors from the previous step
        candidates = []
        for i in range(self.n_features + 1):
            if i == 0:  # Interaction with the state tensor
                current_tensor = state_tensor
            else:  # Interaction with columns of the previous transition tensor
                current_tensor = previous_transition_tensor[:, :, i - 1]

            interaction_tensor = self.interaction_tensors[i]
            current_tensor_expanded = current_tensor.unsqueeze(1).expand(-1, self.n_features, -1)  # Shape: [batch, n_features, n_features]
            interaction_tensor_expanded = interaction_tensor.unsqueeze(0).expand(batch, -1, -1, -1)  # Shape: [batch, n_features, n_features, n_features]
            # Multiply and sum over the last dimension to get the transition tensor
            candidate = torch.einsum('bik,bijk->bij', current_tensor_expanded, interaction_tensor_expanded)
            
            candidates.append(candidate)
            
        candidates_tensor = torch.stack(candidates, dim = -1)
        return candidates_tensor


In [6]:
import torch.nn.functional as F

class SelectorModule(nn.Module):
    def __init__(self, num_slices):
        super(SelectorModule, self).__init__()
        # A simple linear layer to compute importance scores for each slice
        self.importance = nn.Linear(num_slices, num_slices)

    def forward(self, x):
        # Assuming x is of shape [batch_size, height, width, num_slices]
        batch_size, height, width, num_slices = x.shape

        # Flatten the spatial dimensions and compute importance scores
        x_flat = x.view(batch_size, -1, num_slices)  # New shape: [batch_size, height*width, num_slices]
        scores = self.importance(x_flat)  # Computes a score for each slice
        scores = scores.view(batch_size, height, width, num_slices)  # Reshape scores back

        # Apply softmax to get a distribution over slices
        weights = F.softmax(scores, dim=-1)
        
        #print(weights.shape)

        # Use the weights to get a weighted sum of the slices, effectively selecting slices based on importance
        # This step aggregates the slices into a single output per position
        selected = torch.einsum('bhwn,bhwi->bhw', weights, x)
        
        # OVERWRITE LATER ####
        
        selected = x[:,:,:,0]

        return selected

# Example usage
batch_size = 10
tensor = torch.rand(batch_size, 5, 5, 6)  # Example tensor
model = SelectorModule(num_slices=6)

result = model(tensor)
print(result.shape)  # Should print torch.Size([10, 5, 5]), indicating the selection/aggregation step was performed


torch.Size([10, 5, 5, 6])
torch.Size([10, 5, 5])


In [7]:
class CustomLayerExtended(nn.Module):
    def __init__(self, n_features):
        super(CustomLayerExtended, self).__init__()
        self.n_features = n_features
        self.interaction_module = InteractionModule(n_features)
        self.selector_module = SelectorModule(n_features + 1)

    def forward(self, x):
        batch, length, n_features = x.size()
        output = x.new_empty(batch, length, n_features)

        # Initialize previous transition tensors (for the first step)
        # Assuming it's a list of zero tensors for simplicity
        previous_transition_tensor = torch.zeros(batch, n_features, n_features)

        for l in range(length):
            state_tensor = x[:, l, :]
            # Generate candidates
            candidates = self.interaction_module(state_tensor, previous_transition_tensor)
            # Select one candidate
            selected_transition_tensor = self.selector_module(candidates)
            # Update the previous_transition_tensors for the next iteration
            previous_transition_tensor = selected_transition_tensor
            # Compute output for this step
            output[:, l, :] = torch.matmul(selected_transition_tensor, state_tensor.unsqueeze(-1)).squeeze(-1)

        return output, selected_transition_tensor


In [8]:
n_features = 16
batch, length, dim = 13, 64, n_features
x = torch.randn(batch, length, dim)
model = CustomLayerExtended(
    n_features = n_features
)
y = model(x)

print(y[0].shape)
assert y[0].shape == x.shape

torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])


torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13, 16, 16, 17])
torch.Size([13

In [9]:
print(y)

(tensor([[[-1.7505e-01,  2.9085e-01,  4.0809e-01,  ...,  3.5397e-01,
          -5.9970e-01,  4.0113e-01],
         [ 1.2627e+01,  1.2355e+01,  1.1690e+01,  ...,  1.2757e+01,
           1.3073e+01,  1.2994e+01],
         [ 2.9897e+01,  2.9710e+01,  3.3112e+01,  ...,  2.9928e+01,
           3.1684e+01,  2.6141e+01],
         ...,
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan,  ...,         nan,
                  nan,         nan]],

        [[ 1.8559e-01,  1.5199e+00,  8.5425e-02,  ...,  5.5262e-01,
           3.8782e-01,  1.2105e+00],
         [ 2.6192e+01,  2.5515e+01,  2.6083e+01,  ...,  2.6266e+01,
           2.6702e+01,  2.6446e+01],
         [ 3.5918e+02,  3.7913e+02,  3.6455e+02,  ...,  3.5340e+02,
           3.7232e+02,  3.7307e+02],
         ...,
         [        nan,  

# Example Training

## Simple Data Generation

In [10]:
import numpy as np
import torch

def generate_normalized_multivariate_time_series(n_features, total_length, amplitude=1.0):
    t = np.linspace(0, 100 * np.pi, total_length)
    series = np.zeros((total_length, n_features))
    for i in range(n_features):
        series[:, i] = amplitude * np.cos(t * (i + 1) / n_features) + 10
    
    # Normalize such that each timestep's values sum to 1
    series_sum = np.sum(series, axis=1, keepdims=True)
    series_normalized = series / series_sum
    
    return series_normalized

def segment_time_series(series, length):
    # Assuming series is a numpy array of shape [total_length, n_features]
    total_length, n_features = series.shape
    segments = []
    for start in range(0, total_length - length, length):
        segment = series[start:start + length]
        segments.append(segment)
    return np.stack(segments)


In [11]:
n_features = 4
length = 64  # Segment length
total_length = 1024  # Arbitrary total length for the generated series

# Generate and segment the time series
series = generate_normalized_multivariate_time_series(n_features, total_length)
series_x = series[:-1,]
series_y = series[1:,]

segments_x = segment_time_series(series_x, length)
segments_y = segment_time_series(series_y, length)

# Convert to tensors
segments_tensor_x = torch.tensor(segments_x, dtype=torch.float)
segments_tensor_y = torch.tensor(segments_y, dtype=torch.float)

# Prepare inputs and targets
X = segments_tensor_x
# Shift segments to the right by one timestep to create the targets
Y =  segments_tensor_y


In [12]:
from torch import nn, optim

# Model
model = CustomLayerExtended(
    n_features = n_features
)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    # Forward pass
    outputs, _ = model(X)
    loss = criterion(outputs, Y)

    # Backward and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')


torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([15, 4, 4, 5])
torch.Size([

In [13]:
torch.stack([X[1,:,1],Y[1,:,1]], dim = -1)

tensor([[0.2304, 0.2339],
        [0.2339, 0.2378],
        [0.2378, 0.2419],
        [0.2419, 0.2460],
        [0.2460, 0.2497],
        [0.2497, 0.2529],
        [0.2529, 0.2553],
        [0.2553, 0.2569],
        [0.2569, 0.2577],
        [0.2577, 0.2576],
        [0.2576, 0.2570],
        [0.2570, 0.2558],
        [0.2558, 0.2545],
        [0.2545, 0.2530],
        [0.2530, 0.2518],
        [0.2518, 0.2508],
        [0.2508, 0.2502],
        [0.2502, 0.2500],
        [0.2500, 0.2503],
        [0.2503, 0.2511],
        [0.2511, 0.2522],
        [0.2522, 0.2535],
        [0.2535, 0.2549],
        [0.2549, 0.2562],
        [0.2562, 0.2572],
        [0.2572, 0.2577],
        [0.2577, 0.2575],
        [0.2575, 0.2565],
        [0.2565, 0.2546],
        [0.2546, 0.2520],
        [0.2520, 0.2486],
        [0.2486, 0.2447],
        [0.2447, 0.2406],
        [0.2406, 0.2365],
        [0.2365, 0.2327],
        [0.2327, 0.2295],
        [0.2295, 0.2269],
        [0.2269, 0.2253],
        [0.2

In [14]:
import plotly.graph_objects as go

def plot_model_output_vs_target(model_outputs, targets, batch_index=0, feature_index=0):
    # Extract the specified feature for the given batch from both the model outputs and targets
    model_output_series = model_outputs[batch_index, :, feature_index].detach().numpy()
    target_series = targets[batch_index, :, feature_index].numpy()
    
    # Create a range for the x-axis (timesteps)
    timesteps = list(range(model_output_series.shape[0]))
    
    # Create traces
    model_trace = go.Scatter(x=timesteps, y=model_output_series, mode='lines', name='Model Output')
    target_trace = go.Scatter(x=timesteps, y=target_series, mode='lines', name='Target')
    
    # Create the figure and add traces
    fig = go.Figure()
    fig.add_trace(model_trace)
    fig.add_trace(target_trace)
    
    # Add title and labels
    fig.update_layout(title=f'Model Output vs Target for Feature {feature_index}, Batch {batch_index}',
                      xaxis_title='Timestep',
                      yaxis_title='Value')
    
    # Show the figure
    fig.show()

# Assuming `y` and `Y` are your model outputs and targets, respectively
# Adjust batch_index and feature_index as needed
plot_model_output_vs_target(outputs, Y, batch_index=2, feature_index=2)
